<a href="https://colab.research.google.com/github/Gerilda/Arrhythmia/blob/main/ptbxl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install wfdb

     |████████████████████████████████| 133kB 7.8MB/s 


In [ ]:
import pandas as pd
import io
from google.colab import files

uploaded = files.upload()
Y = pd.read_csv(io.BytesIO(uploaded['ptbxl_database.csv']), index_col='ecg_id')

uploaded = files.upload()
agg_df = pd.read_csv(io.BytesIO(uploaded['scp_statements.csv']), index_col=0)

KeyboardInterrupt: ignored

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

# Y = pd.read_csv('gdrive/My Drive/scp_statements.csv', index_col='ecg_id')

path = 'gdrive/My Drive/Colab Notebooks/ptb/'
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)

Mounted at /content/gdrive


In [4]:
# import pandas as pd
import numpy as np
import wfdb
import ast

def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

# path = 'C:/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1/'
sampling_rate=100

# load and convert annotation data

# uploaded = files.upload()

# Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
# Y = pd.read_csv(io.BytesIO(uploaded['ptbxl_database.csv']), index_col='ecg_id')

Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)

# Load scp_statements.csv for diagnostic aggregation
# uploaded = files.upload()

# agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
# agg_df = pd.read_csv(io.BytesIO(uploaded['scp_statements.csv']), index_col=0)

agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

# Split data into train and test
test_fold = 10
# Train
X_train = X[np.where(Y.strat_fold != test_fold)]
y_train = Y[(Y.strat_fold != test_fold)].diagnostic_superclass
# Test
X_test = X[np.where(Y.strat_fold == test_fold)]
y_test = Y[Y.strat_fold == test_fold].diagnostic_superclass

In [5]:
X_train

array([[[-1.190e-01, -5.500e-02,  6.400e-02, ..., -2.600e-02,
         -3.900e-02, -7.900e-02],
        [-1.160e-01, -5.100e-02,  6.500e-02, ..., -3.100e-02,
         -3.400e-02, -7.400e-02],
        [-1.200e-01, -4.400e-02,  7.600e-02, ..., -2.800e-02,
         -2.900e-02, -6.900e-02],
        ...,
        [ 6.900e-02,  0.000e+00, -6.900e-02, ...,  2.400e-02,
         -4.100e-02, -5.800e-02],
        [ 8.600e-02,  4.000e-03, -8.100e-02, ...,  2.420e-01,
         -4.600e-02, -9.800e-02],
        [ 2.200e-02, -3.100e-02, -5.400e-02, ...,  1.430e-01,
         -3.500e-02, -1.200e-01]],

       [[ 4.000e-03,  1.380e-01,  1.340e-01, ...,  1.920e-01,
          8.300e-02,  8.800e-02],
        [-2.000e-02,  1.160e-01,  1.360e-01, ...,  1.560e-01,
          5.700e-02,  6.300e-02],
        [-5.300e-02,  9.200e-02,  1.450e-01, ...,  1.070e-01,
          1.300e-02,  2.200e-02],
        ...,
        [ 1.210e-01,  3.980e-01,  2.770e-01, ..., -1.065e+00,
         -4.920e-01, -1.560e-01],
        [-3.

In [7]:
y_train

ecg_id
1        [NORM]
2        [NORM]
3        [NORM]
4        [NORM]
5        [NORM]
          ...  
21833    [STTC]
21834    [NORM]
21835    [STTC]
21836    [NORM]
21837    [NORM]
Name: diagnostic_superclass, Length: 19634, dtype: object